# The time to beat is 60-80 seconds per file.

In [1]:
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import pandas as pd
from metatlas import metatlas_objects as metob
from metatlas.helpers import dill2plots as dp
import numpy as np
import time
import copy
import dill
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)

('Metatlas live in ', '/global/homes/b/bpb/metatlas/metatlas')
you're running on fb2ee5119e85 at 172.17.0.2 


In [22]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%nyogilipidsA%pos%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['QC','Blank','extractioncontrol'])

31
0 20160504_KBL_NyogiLipidsA_POS_24hr_BD_controlBD_noglc 2016-07-18 19:26:28
1 20160504_KBL_NyogiLipidsA_POS_48hr_BD_treatmentBD_glc 2016-07-18 19:26:04
2 20160504_KBL_NyogiLipidsA_POS_24hr_BD_treatmentBD_glc 2016-07-18 19:26:05
3 20160504_KBL_NyogiLipidsA_POS_72hr_BD_controlBD_noglc 2016-07-18 19:26:19
4 20160504_KBL_NyogiLipidsA_POS_PC_D31__ 2016-07-18 19:26:05
5 20160504_KBL_NyogiLipidsA_POS_48hr_BD_controlBD_noglc 2016-07-18 19:26:06
6 20160504_KBL_NyogiLipidsA_POS_84hr0hr_BDBG_treatmentBD_glc 2016-07-18 19:26:10
7 20160504_KBL_NyogiLipidsA_POS_SQDG_nolabel__ 2016-07-18 19:26:10
8 20160504_KBL_NyogiLipidsA_POS_48hr_BG_controlBG_glcglc 2016-07-18 19:26:07
9 20160504_KBL_NyogiLipidsA_POS_DGTS_D9__ 2016-07-18 19:26:11
10 20160504_KBL_NyogiLipidsA_POS_lysoPC_17oddchain__ 2016-07-18 19:26:13
11 20160504_KBL_NyogiLipidsA_POS_DGDG_nolabel__ 2016-07-18 19:26:18
12 20160504_KBL_NyogiLipidsA_POS_24hr_BG_treatmentBG_glcnoglc 2016-07-18 19:26:36
13 20160504_KBL_NyogiLipidsA_POS_MGDG_nolabel_

# Functions

In [23]:
def df_container_from_metatlas_file(my_file):
    data_df = pd.DataFrame()
    pd_h5_file  = pd.HDFStore(my_file.hdf5_file)
    keys = pd_h5_file.keys()
    pd_h5_file.close()
    df_container = {}
    for k in keys:
        if ('ms2' in k) and not ('_mz' in k):
            new_df = pd.read_hdf(my_file.hdf5_file,k)
            df_container[k[1:]] = new_df
    return df_container


In [24]:
import os
df = pd.DataFrame()
for group in groups:
    for my_file in group.items:
        container = df_container_from_metatlas_file(my_file)
        container['ms2_pos']['hdf5_file'] = os.path.basename(my_file.hdf5_file)
        df = pd.concat([df,container['ms2_pos']],axis=0)


In [25]:
df.head()

,mz,i,rt,precursor_MZ,precursor_intensity,collision_energy,hdf5_file
0,59.184872,1954.242432,0.011703,413.265106,143063.640625,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5
1,61.079731,2154.174805,0.011703,413.265106,143063.640625,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5
2,71.000175,2465.985107,0.011703,413.265106,143063.640625,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5
3,94.132423,2286.211182,0.011703,413.265106,143063.640625,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5
4,171.005829,3223.530518,0.011703,413.265106,143063.640625,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5


In [26]:
grouped = df.groupby(['precursor_MZ','rt','precursor_intensity','collision_energy','hdf5_file']).aggregate(lambda x: tuple(x))
grouped.mz = grouped.mz.apply(list)
grouped.i = grouped.i.apply(list)
grouped = grouped.reset_index()
grouped['spectrum'] = map(lambda x,y:(x,y),grouped['mz'],grouped['i'])
grouped['spectrum'] = grouped['spectrum'].apply(lambda x: zip(x[0],x[1]))
grouped.drop(['mz','i'], axis=1, inplace=True)


In [27]:
grouped.head()

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum
0,80.049644,1.029684,5.853951e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_19_48hr_BD_c_treatmentBD_glc___Run82.h5,"[(53.0031, 147646.0), (53.0392, 7690.6), (53.9981, 6578.45), (54.0066, 1333.97), (63.5309, 1273...."
1,80.049644,7.208591,1.073722e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_11_24hr_BD_c_treatmentBD_glc___Run85.h5,"[(50.0159, 1531.24), (51.0238, 10235.2), (53.0031, 316264.0), (53.0394, 58215.1), (53.9983, 1712..."
2,80.049652,4.948564,1.116939e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_55_48hr_BG_c_controlBG_glcglc___Run91.h5,"[(51.0237, 3196.43), (53.0031, 295493.0), (53.0395, 34660.0), (53.9983, 15183.4), (54.0064, 6547..."
3,80.049652,7.676896,1.179940e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_39_12hr_BG_c_controlBG_glcglc___Run97.h5,"[(51.0238, 13222.7), (52.0316, 1647.83), (52.3341, 1239.14), (53.003, 322221.0), (53.0394, 64183..."
4,80.049660,0.765142,5.261536e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_3_0hr_BD_c_controlBD_noglc___Run88.h5,"[(53.0031, 137306.0), (53.0393, 8933.92), (53.9981, 7395.19), (55.2099, 1419.99), (56.07, 1265.7..."


In [28]:
target = 184.0739
min_intensity = 1e4
ppm = 10
def search_spectrum(spectrum,target,min_intensity,ppm):
    for s in spectrum:
        if (abs(s[0] - target) < target*ppm/1e6) & (s[1] > min_intensity):
            return True
    return False
grouped['has_target_fragment'] = grouped['spectrum'].apply(search_spectrum,args = (target,min_intensity,ppm))



In [29]:
idx = (grouped.has_target_fragment) & (grouped.rt > 3) & (grouped.rt < 7) & (grouped.precursor_MZ > 500)
print len(idx)
df_sqdg = grouped[idx]

321174


In [30]:
df_sqdg = df_sqdg.sort_values('precursor_intensity',ascending=False)

In [31]:
df_sqdg.head(10)

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum,has_target_fragment
282224,734.590271,5.937315,588514240.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_6_24hr_BD_b_controlBD_noglc___Run181.h5,"[(53.1054, 284154.0), (58.066, 3.11622e+06), (59.0739, 317586.0), (60.0816, 635078.0), (62.0885,...",True
285883,760.605652,6.011604,522524480.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_7_24hr_BD_c_controlBD_noglc___Run112.h5,"[(53.7186, 328855.0), (58.066, 3.41337e+06), (58.643, 382559.0), (60.0818, 510233.0), (60.3059, ...",True
285850,760.605164,6.052485,510743552.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_27_84hr_BD_a_controlBD_noglc___Run79.h5,"[(53.2064, 430968.0), (58.0659, 2.4414e+06), (60.0815, 764318.0), (88.9851, 426366.0), (113.06, ...",True
285919,760.606140,6.030255,475722080.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_23_72hr_BD_c_controlBD_noglc___Run109.h5,"[(57.0343, 305309.0), (58.0659, 2.41338e+06), (58.2284, 357781.0), (59.8465, 331881.0), (60.0817...",True
282280,734.590881,5.924044,470385344.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_14_48hr_BD_b_controlBD_noglc___Run178.h5,"[(53.2342, 293072.0), (58.0659, 2.18239e+06), (59.5515, 290735.0), (60.0815, 1.43479e+06), (64.3...",True
282226,734.590271,5.960624,468064864.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_3_0hr_BD_c_controlBD_noglc___Run88.h5,"[(51.4845, 281546.0), (51.6861, 192978.0), (58.0659, 1.6516e+06), (60.0817, 518083.0), (67.6236,...",True
285915,760.606140,5.997443,466445568.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_1_0hr_BD_a_controlBD_noglc___Run28.h5,"[(58.0659, 1.31199e+06), (60.0813, 483575.0), (76.0236, 309495.0), (87.4467, 335823.0), (120.041...",True
285865,760.605469,6.051013,460366208.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run43.h5,"[(56.4215, 307328.0), (58.0659, 1.96104e+06), (60.0815, 557236.0), (65.4694, 323003.0), (67.5032...",True
285891,760.605713,6.003944,460229056.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_5_24hr_BD_a_controlBD_noglc___Run31.h5,"[(53.4194, 285966.0), (58.0659, 2.30004e+06), (60.0815, 599418.0), (65.8736, 302721.0), (67.383,...",True
285911,760.606079,6.030700,433380512.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_15_48hr_BD_c_controlBD_noglc___Run115.h5,"[(58.0661, 2.4865e+06), (58.872, 260870.0), (60.9256, 340645.0), (111.504, 410569.0), (116.391, ...",True


In [32]:
df_unique_sqdg = pd.DataFrame()
df_unique_sqdg = df_sqdg.iloc[[0]]
for i in df_sqdg.index:
    idx = abs(df_sqdg.loc[i,'precursor_MZ'] - df_unique_sqdg.precursor_MZ) < 0.02
    if sum(idx) == 0:
        #The mz is new so add it
        df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)
    else:
        #the mz has been seen before; so see if the rt is new
        if sum(abs(df_unique_sqdg[idx].rt - df_sqdg.loc[i,'rt']) < 0.1)==0:
            df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)

In [33]:
df_unique_sqdg.shape

(268, 7)

In [34]:
df_unique_sqdg.head(10)

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum,has_target_fragment
282224,734.590271,5.937315,588514240.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_6_24hr_BD_b_controlBD_noglc___Run181.h5,"[(53.1054, 284154.0), (58.066, 3.11622e+06), (59.0739, 317586.0), (60.0816, 635078.0), (62.0885,...",True
285883,760.605652,6.011604,522524480.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_7_24hr_BD_c_controlBD_noglc___Run112.h5,"[(53.7186, 328855.0), (58.066, 3.41337e+06), (58.643, 382559.0), (60.0818, 510233.0), (60.3059, ...",True
285663,760.369995,5.983567,348493184.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_48_24hr_BG_d_controlBG_glcglc___Run175.h5,"[(117.631, 209881.0), (144.102, 6.04292e+06), (152.213, 445135.0), (162.112, 2.29687e+06), (184....",True
285410,758.589539,5.674184,270865568.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_28_84hr_BD_b_controlBD_noglc___Run187.h5,"[(56.4435, 198692.0), (57.1589, 200592.0), (58.066, 1.97225e+06), (59.0737, 360716.0), (60.0816,...",True
285835,760.604980,6.125972,221177488.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_49_48hr_BG_c_treatmentBG_glcnoglc___Run67.h5,"[(57.2595, 157702.0), (58.0659, 1.26213e+06), (60.0815, 232120.0), (68.5156, 166186.0), (70.1307...",True
282039,732.574463,5.603048,214612272.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_20_48hr_BD_d_treatmentBD_glc___Run154.h5,"[(55.3163, 86106.8), (58.0661, 992232.0), (60.0816, 235149.0), (67.0547, 143415.0), (81.0705, 11...",True
285522,758.590149,5.567500,178740224.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_40_12hr_BG_d_controlBG_glcglc___Run196.h5,"[(58.066, 1.58684e+06), (59.0739, 203448.0), (60.0816, 270974.0), (67.0549, 289580.0), (80.0975,...",True
288913,782.566467,5.685224,167832560.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_1_0hr_BD_a_controlBD_noglc___Run28.h5,"[(56.5031, 140579.0), (58.0661, 562305.0), (59.0741, 174680.0), (60.0816, 2.18447e+06), (71.0734...",True
288830,781.580017,5.698529,163624144.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_2_0hr_BD_b_controlBD_noglc___Run118.h5,"[(124.012, 53563.5), (125.0, 1.61201e+06), (163.819, 67580.8), (173.494, 165596.0), (173.509, 21...",True
289213,784.581909,6.112022,162859040.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_39_12hr_BG_c_controlBG_glcglc___Run97.h5,"[(55.6292, 108249.0), (56.7685, 120312.0), (58.0663, 583933.0), (59.0739, 234454.0), (60.0815, 1...",True


In [35]:
df_unique_sqdg.to_csv('/global/homes/b/bpb/Downloads/pc_potential_hits.csv')

In [36]:
from matplotlib import pyplot as plt
%matplotlib inline

In [37]:
print(plt.style.available)

[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']


In [38]:
for i in df_unique_sqdg.index:
    s = df_unique_sqdg.loc[i,'spectrum']
    mz = zip(*s)[0]
    intensity = zip(*s)[1]
    fig = plt.figure(figsize=(11,8.5))
    plt.style.use('seaborn-ticks')
    ax = plt.gca()
    ax.vlines(mz,intensity*0,intensity)
    ax.set_xlim(0,df_unique_sqdg.loc[i,'precursor_MZ']+5)
    ax.set_xlabel('m/z')
    ax.set_ylabel('intensity')
    sx = np.argsort(intensity)[::-1]
    labels = [1.001e9]
    for i_mz in sx:
        if np.min(np.abs(mz[i_mz] - labels)) > 0.1 and intensity[i_mz] > 0.02 * np.max(intensity):
            ax.annotate('%5.4f'%mz[i_mz], xy=(mz[i_mz], 1.01*intensity[i_mz]),rotation = 90, horizontalalignment = 'center', verticalalignment = 'left')
            labels.append(mz[i_mz])
    plt.tight_layout()
    L = plt.ylim()
    plt.ylim(L[0],L[1]*1.12)
    fig.savefig('/global/homes/b/bpb/Downloads/pc_msms/msms_spectra_%d.png'%i)
    plt.close(fig)
